In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from TaxiDataFormatter.dataformatter import formatter

fig=plt.figure(figsize=(16, 9), dpi= 80, facecolor='w', edgecolor='k')
%matplotlib inline

In [2]:
!ls

TaxiDataFormatter     sample_submission.zip test.zip
Taxi_v1.ipynb         sub_1.csv             train.csv
Taxi_v2.ipynb         sub_2.csv             train.zip
sample_submission.csv test.csv


# Load Data and Analyze

In [3]:
train = pd.read_csv('train.csv', delimiter=',')
test = pd.read_csv('test.csv', delimiter=',')

In [4]:
train.shape

(1458644, 11)

In [5]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [6]:
y = train[['trip_duration']].copy()
Y = np.log(y+1)
train = train.drop(['trip_duration','dropoff_datetime'],axis=1)

In [7]:
%%timeit
mod_df1 = formatter(train)
mod_df1 = mod_df1.drop(['id'],axis=1)
mod_df1.head()
X = mod_df1.values
X.shape

,passenger_count,dist,flag_n,flag_y,vendor_1,vendor_2,p_time,pickup_weekday
0,1,1.498991,1,0,0,1,17,1
1,1,1.806074,1,0,1,0,0,7
2,1,6.387103,1,0,0,1,11,2
3,1,1.485965,1,0,0,1,19,3
4,1,1.188962,1,0,0,1,13,6


(1458644, 8)

In [8]:
test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [11]:
%%time
mod_df2 = formatter(test)
mod_df2 = mod_df2.drop(['id'],axis=1)
print(mod_df2.head())
X_test = mod_df2.values

   passenger_count      dist  flag_n  flag_y  vendor_1  vendor_2  p_time  \
0                1  2.747288       1       0         1         0      23   
1                1  2.760105       1       0         1         0      23   
2                1  1.306565       1       0         1         0      23   
3                1  5.270742       1       0         0         1      23   
4                1  0.961143       1       0         1         0      23   

   pickup_weekday  
0               4  
1               4  
2               4  
3               4  
4               4  
CPU times: user 33.3 s, sys: 446 ms, total: 33.8 s
Wall time: 28.4 s


# Split the data

In [12]:
%%time
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(X)
x_test = sc.transform(X_test)

CPU times: user 951 ms, sys: 340 ms, total: 1.29 s
Wall time: 904 ms


In [13]:
from sklearn.model_selection import train_test_split
xtr, xtv, ytr, ytv = train_test_split(x_train,Y,test_size=0.05, random_state=43)

In [14]:
from sklearn.metrics import mean_squared_error

# Build the model

In [12]:
#build your model xgb regressor
from sklearn.linear_model import LinearRegression

In [13]:

import lightgbm as lgb

d_train = lgb.Dataset(x_train, label=y_train)

params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10

clf = lgb.train(params, d_train, 100)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [14]:
y_train_hat = model.predict(xtr)
y_test_hat = model.predict(xtv)

train_rmse = mean_squared_error(ytr, y_train_hat)
test_rmse = mean_squared_error(ytv, y_test_hat)
print("train Error: ",train_rmse)
print("test Error: ",test_rmse)
t2 = time.time()
print(t2-t1)

train Error:  0.404596389851222
test Error:  0.46008128209892635
0.7989637851715088


# Load Test Data

In [15]:
submission = pd.read_csv('sample_submission.csv')
submission.head()

,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959


In [16]:
yt_hat = model.predict(X_test)

In [17]:
submission['trip_duration'] = np.exp(yt_hat)

In [18]:
submission.head()

,id,trip_duration
0,id3004672,549.741541
1,id3505355,594.104033
2,id1217141,494.804432
3,id2150126,794.565489
4,id1598245,464.783193


In [19]:
submission.to_csv('sub_2.csv',index=False)